In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.preprocessing.image import load_img
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
import os
import numpy as np
from tqdm.notebook import tqdm

In [2]:
train_dir = r'C:\Users\harkr\OneDrive\Documents\EMOTION DETECTION(PYTHON)\Train'
test_dir = r'C:\Users\harkr\OneDrive\Documents\EMOTION DETECTION(PYTHON)\test'

In [3]:
def create_df(dir):
    image = []
    labels = []
    for label in os.listdir(dir):
        for imgname in os.listdir(os.path.join(dir, label)):
            image.append(os.path.join(dir, label, imgname))
            labels.append(label)
        print(label)
    return image, labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = create_df(train_dir)

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = create_df(test_dir)

In [ ]:
print(train)
print(test)

In [7]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [ ]:
train_features = extract_features(train['image'])

In [ ]:
test_features = extract_features(test['image'])

In [10]:
x_train = train_features/255.0
x_test = test_features/255.0

In [11]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

In [13]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [14]:
from keras.utils import to_categorical

In [15]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3, 3), padding='same',
          activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Dropout(0.3))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))

In [17]:
model.compile(optimizer='rmsprop', loss='crossentropy', metrics=['accuracy'])

In [19]:
model.save('emotion_models.keras')